# PheWAS Analysis of FTO Variant rs8050136

This notebook performs the PheWAS analysis of our extracted variant. The 16 cpu + 104gb ram is sufficient for this single variant analysis, though it will take around 20 minutes per analysis. This PheWAS is the simplest form- we are not stratifying on genomic ancestry here, which may be wise for more scientifically robust anaylses.

## Load libraries and data

In [ ]:
library(PheWAS)

In [ ]:
library(tidyverse)

### Covariates

From exercises 3, 4, and 5.

In [ ]:
covariates = read_csv("covariates.csv")
covariates %>% transmute(across(-person_id, \(x) sample(x))) %>% head()
#head(covariates) 

### Variant information

From exercises 1 and 2.

In [ ]:
snp = read_delim("fto_snp.raw", delim=" ")  %>% transmute(person_id=IID, rs8050136_A=`chr16:53782363:C:A_A`)
snp %>% transmute(across(-person_id, \(x) sample(x))) %>% head()
#head(snp)

### Phecode phenotypes

From exercise 3.

In [ ]:
phenotypes = read_csv("phecode_table.csv")
phenotypes[,1:10] %>% transmute(across(-person_id, \(x) sample(x))) %>% head()
#head(phenotypes)

### Merge the tables for analysis

In [ ]:
data = inner_join(inner_join(covariates, snp), phenotypes)

In [ ]:
dim(data)

## Run the PheWAS

This will execute our PheWAS analysis. We will use every phecode phenotype (The names of our phenotypes table, except person_id) and use a few covariates. This analysis uses all 16 cores.

In [ ]:
results = phewas_ext(data, phenotypes = names(phenotypes)[-1], 
                     genotypes=c("rs8050136_A"),
                    covariates=c("sex_ab_F", "age", "pc1", "pc2", "pc3"),
                    cores=32)

### Plot Results

I find these plot options yield an easy, nice looking plot in the notebook environment.

In [ ]:
options(repr.plot.width=15, repr.plot.height=8)

In [ ]:
phewasManhattan(results, OR.direction=T, title ="PheWAS against rs8050136_A")

### Show results tables

The phewasDT() function creates a handy interactive table. It won't render in the workbench preview, but is nice when you are looking at results.

In [ ]:
phewasDT(results)

### Static results table

This code gives a simple table view of top results.

In [ ]:
p.digits=3
OR.digits=2
results %>% addPhecodeInfo() %>% filter(!is.na(p)) %>% arrange(p) %>% 
        transmute(phenotype, description, snp, OR = round(OR, 
            OR.digits), p = signif(p, p.digits), n_total, n_cases, 
            n_controls) %>% head()

## BMI-Adjusted PheWAS Analysis

We can also adjust for BMI in our analysis. The next section does the same type of analysis, but adjusted for BMI. 

In [ ]:
results_adj = phewas_ext(data, phenotypes = names(phenotypes)[-1], 
                     genotypes=c("rs8050136_A"),
                    covariates=c("sex_ab_F", "age", "pc1", "pc2", "pc3", "BMI"),
                    cores=32)

In [ ]:
phewasManhattan(results_adj, OR.direction=T, title ="PheWAS against rs8050136_A, BMI-Adjusted",
               y.axis.interval=1)

In [ ]:
phewasDT(results_adj)

In [ ]:
results_adj %>% addPhecodeInfo() %>% filter(!is.na(p)) %>% arrange(p) %>% 
        transmute(phenotype, description, snp, OR = round(OR, 
            OR.digits), p = signif(p, p.digits), n_total, n_cases, 
            n_controls) %>% head()

## Save our results

We can save our results table and download it, as it does not contain participant-level information.

In [ ]:
write_csv(results_adj, file="adjusted_results.csv")